In [1]:
import pandas as pd
import requests
import urllib3
import ssl
import io

In [2]:
def format_oecd_data(df):
    df_formatted = pd.DataFrame(index=df['TIME'].unique())

    for country in df['Country'].unique():
        df_formatted[country] = df[df['Country'] == country]['Value']

    return df_formatted

In [3]:
def check_data_coverage(df):
    print('Missing first:')
    for country in df.columns:
        first_valid = df[country].first_valid_index()
        if not first_valid.startswith('1999'):
            print(country, first_valid)

    print('Missing last:')
    for country in df.columns:
        last_valid = df[country].last_valid_index()
        if not last_valid.startswith('2023'):
            print(country, last_valid)
    print('Done')

In [4]:
class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

In [5]:
countries = 'AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19'

def get_oecd_data(indicator, subject, measure, freq, countries_before_subject=True, with_measure=True):
    if freq == 'Q':
        startTime = '1999-Q1'
    else:
        startTime = '1999-01'

    if countries_before_subject:
        countries_and_subject = f'{countries}.{subject}'
    else:
        countries_and_subject = f'{subject}.{countries}'
    
    if with_measure:
        url = f'https://stats.oecd.org/SDMX-JSON/data/{indicator}/{countries_and_subject}.{measure}.{freq}/all?startTime={startTime}&contentType=csv'
    else:
        url = f'https://stats.oecd.org/SDMX-JSON/data/{indicator}/{countries_and_subject}.{freq}/all?startTime={startTime}&contentType=csv'

    print(url)
    res = get_legacy_session().get(url)
    df = pd.read_csv(io.BytesIO(res.content), sep=',')
    df.index = df['TIME']
    
    df_formatted = format_oecd_data(df)
    df_formatted = df_formatted.sort_index()
    check_data_coverage(df_formatted)
    df_formatted.to_csv(f'OECD_{indicator}_{subject}.csv')
    return df_formatted

In [7]:
indicator = 'QNA' # Quaterly National Accounts
subject = 'B1_GE' # Gross domestic product - expenditure approach
measure = 'GYSA' # Growth rate based on seasonally adjusted volume data, percentage change on the same quarter the previous year
freq = 'Q' # Quaterly

df = get_oecd_data(indicator, subject, measure, freq, countries_before_subject=True)
df

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.B1_GE.GYSA.Q/all?startTime=1999-Q1&contentType=csv
Missing first:
Russia 2004-Q1
Saudi Arabia 2011-Q1
Colombia 2006-Q1
Missing last:
Russia 2021-Q3
Done


,Australia,Austria,Belgium,Canada,Czech Republic,Denmark,Finland,France,Germany,Greece,...,Brazil,Chile,China (People's Republic of),India,Indonesia,Israel,Russia,Saudi Arabia,South Africa,Colombia
1999-Q1,5.173796,2.765020,1.904799,4.194104,0.395327,2.311058,5.934263,2.784202,0.793797,2.361704,...,0.680309,-2.237507,8.9,6.19852,-6.407874,1.047212,NaN,NaN,0.979536,NaN
1999-Q2,4.541221,3.085828,2.956681,5.009661,0.575203,3.621945,4.390232,2.617900,1.220621,2.861976,...,-0.519584,-3.605963,7.9,7.53136,1.497007,4.918468,NaN,NaN,1.639478,NaN
1999-Q3,3.776954,4.049713,4.147585,5.690714,1.473288,2.381317,3.644918,3.529290,1.859483,2.896443,...,-0.332025,-0.906906,7.6,6.92997,2.921313,4.176798,NaN,NaN,2.974826,NaN
1999-Q4,3.962759,3.967943,5.145646,5.738707,2.542947,3.481499,3.610436,4.179188,3.124281,3.724339,...,2.063191,5.105253,6.7,7.28888,5.991807,5.759049,NaN,NaN,4.007388,NaN
2000-Q1,3.623640,4.246926,4.798243,5.516373,3.879361,3.552325,4.969214,4.357079,3.399247,3.330422,...,3.004970,6.077906,8.7,7.79075,3.483446,7.018639,NaN,NaN,4.221428,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,3.092582,6.556554,4.122701,4.719747,3.359600,3.151989,2.610236,3.875825,1.555184,7.123086,...,3.583579,5.499798,0.4,13.10546,5.288928,7.733142,NaN,10.855582,0.150881,12.151756
2022-Q3,5.994062,1.883439,2.054921,3.818950,1.350071,2.284758,1.087909,1.180414,1.245724,4.050557,...,3.619418,0.007175,3.9,5.51228,6.111392,5.923477,NaN,7.629264,3.875444,7.332106
2022-Q4,2.696466,2.738487,1.527137,2.074066,0.120022,0.372316,-0.456465,0.719408,0.824963,4.754101,...,2.546816,-2.137102,2.9,4.63865,5.157855,2.651027,NaN,6.499575,1.347491,1.970428
2023-Q1,2.444471,1.848958,1.279136,2.079295,-0.422889,2.029691,-0.078421,0.826340,-0.268399,1.968942,...,3.437365,-0.957612,4.5,6.27858,4.940213,4.022579,NaN,2.186150,0.224079,2.998554


In [8]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'IRSTCI01' # Overnight Interbank Rate
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, countries_before_subject=False)
df

https://stats.oecd.org/SDMX-JSON/data/KEI/IRSTCI01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Missing first:
Missing last:
Sweden 2020-10
Russia 2022-08
Done


,Australia,Austria,Belgium,Canada,Czech Republic,Denmark,Finland,France,Germany,Greece,...,Brazil,Chile,China (People's Republic of),India,Indonesia,Israel,Russia,South Africa,Colombia,Euro area (19 countries)
1999-01,4.75,3.137000,3.137000,4.991400,9.09,3.7699,3.137000,3.137000,3.137000,11.425105,...,38.25,12.94,4.59,9.00,36.43,14.55,60.0,18.830000,26.98,3.137000
1999-02,4.75,3.118000,3.118000,4.995700,8.00,3.5736,3.118000,3.118000,3.118000,10.180684,...,41.00,6.25,4.59,9.00,37.50,14.56,60.0,17.360000,23.05,3.118000
1999-03,4.75,2.925200,2.925200,4.994300,7.70,3.4139,2.925200,2.925200,2.925200,10.147409,...,46.00,6.84,4.59,8.00,37.84,14.00,60.0,16.500000,22.14,2.925200
1999-04,4.75,2.709100,2.709100,4.777100,7.27,3.0343,2.709100,2.709100,2.709100,10.151450,...,32.00,12.81,4.59,8.00,35.19,13.44,60.0,15.660000,20.72,2.709100
1999-05,4.75,2.551000,2.551000,4.592900,6.91,2.8990,2.551000,2.551000,2.551000,10.402900,...,23.50,11.77,4.59,8.00,28.73,12.86,60.0,15.460000,18.92,2.551000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,3.60,2.899081,2.899081,4.491718,7.00,2.7100,2.899081,2.899081,2.899081,2.899081,...,13.75,11.25,2.90,6.75,5.63,4.50,NaN,7.750000,13.00,2.899081
2023-05,3.83,3.046309,3.046309,4.492648,7.00,2.9000,3.046309,3.046309,3.046309,3.046309,...,13.75,11.25,2.90,6.75,5.56,4.56,NaN,7.840909,13.28,3.046309
2023-06,4.05,3.204590,3.204590,4.683945,7.00,3.0900,3.204590,3.204590,3.204590,3.204590,...,13.75,11.25,2.90,6.75,5.61,4.75,NaN,8.250000,13.28,3.204590
2023-07,4.10,3.403263,3.403263,4.903890,7.00,3.2300,3.403263,3.403263,3.403263,3.403263,...,13.75,11.25,NaN,6.75,5.61,4.75,NaN,8.250000,13.26,3.403263


In [9]:
subject = 'IR3TIB01' # 3 month interbank rate
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, countries_before_subject=False)
df

https://stats.oecd.org/SDMX-JSON/data/KEI/IR3TIB01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Missing first:
Japan 2002-04
India 2011-12
Missing last:
Türkiye 2008-04
Russia 2022-02
Done


,Australia,Austria,Belgium,Canada,Czech Republic,Denmark,Finland,France,Germany,Greece,...,United States,Chile,China (People's Republic of),Indonesia,Israel,Russia,South Africa,Colombia,Euro area (19 countries),India
1999-01,4.79,3.1321,3.1321,5.004500,8.49,3.887500,3.1321,3.1321,3.1321,11.690000,...,4.89,12.75,4.5161,45.50,13.362381,43.9,16.404800,33.23,3.1321,NaN
1999-02,4.77,3.0934,3.0934,5.033500,8.18,3.615000,3.0934,3.0934,3.0934,10.664211,...,4.90,10.04,6.2767,38.20,13.348500,23.3,15.542500,30.25,3.0934,NaN
1999-03,4.81,3.0467,3.0467,5.057391,7.58,3.459022,3.0467,3.0467,3.0467,10.050455,...,4.91,11.09,4.3972,34.85,12.811053,21.6,14.623846,25.32,3.0467,NaN
1999-04,4.76,2.6965,2.6965,4.716667,7.09,3.159773,2.6965,2.6965,2.6965,10.084500,...,4.88,13.59,4.3383,34.09,12.308750,20.6,13.753043,22.08,2.6965,NaN
1999-05,4.86,2.5790,2.5790,4.615500,6.96,3.108452,2.5790,2.5790,2.5790,9.945000,...,4.92,11.71,4.2458,31.20,12.021000,21.0,13.516800,19.01,2.5790,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,3.66,3.1670,3.1670,4.799086,7.18,3.281571,3.1670,3.1670,3.1670,3.167000,...,5.03,NaN,2.8900,6.76,4.290000,NaN,7.718824,12.53,3.1670,6.81
2023-05,3.90,3.3664,3.3664,4.788520,7.18,3.396137,3.3664,3.3664,3.3664,3.366400,...,5.15,NaN,2.9800,6.76,4.360000,NaN,8.043182,12.57,3.3664,6.87
2023-06,4.25,3.5359,3.5359,5.021803,7.15,3.562376,3.5359,3.5359,3.5359,3.535900,...,5.22,NaN,2.8100,6.75,4.560000,NaN,8.485714,13.02,3.5359,6.76
2023-07,4.30,3.6718,3.6718,5.219326,7.11,3.720800,3.6718,3.6718,3.6718,3.671800,...,5.35,NaN,2.5700,6.75,4.600000,NaN,8.567619,13.58,3.6718,6.72


In [10]:
subject = 'IRLTLT01' # Long-term interest rate
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, countries_before_subject=False)
df

https://stats.oecd.org/SDMX-JSON/data/KEI/IRLTLT01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Missing first:
Czech Republic 2000-04
Korea 2000-10
Mexico 2001-07
Poland 2001-01
Chile 2004-07
Colombia 2003-01
India 2011-12
Indonesia 2012-01
China (People's Republic of) 2014-01
Missing last:
Russia 2018-06
Done


,Australia,Austria,Belgium,Canada,Czech Republic,Denmark,Finland,France,Germany,Greece,...,Chile,Israel,Russia,South Africa,Euro area (19 countries),Colombia,India,Indonesia,Brazil,China (People's Republic of)
1999-01,5.127895,3.8426,3.90,4.946500,NaN,4.035,3.910000,3.7700,3.702000,6.30,...,NaN,5.29,96.34,15.890000,3.8220,NaN,NaN,NaN,12.84,NaN
1999-02,5.300000,3.9840,4.04,5.124000,NaN,4.188,4.040000,3.9290,3.849000,6.00,...,NaN,5.29,99.47,14.880000,3.9742,NaN,NaN,NaN,12.84,NaN
1999-03,5.526522,4.1648,4.26,5.220435,NaN,4.432,4.260000,4.1335,4.037826,6.00,...,NaN,5.05,98.67,14.540000,4.1810,NaN,NaN,NaN,12.84,NaN
1999-04,5.357500,4.0321,4.11,5.029524,NaN,4.217,4.070000,3.9762,3.850000,5.90,...,NaN,5.08,110.55,14.580000,4.0407,NaN,NaN,NaN,13.48,NaN
1999-05,5.875714,4.2107,4.29,5.359500,NaN,4.461,4.240000,4.1600,4.012105,5.80,...,NaN,5.05,96.75,15.140000,4.2049,NaN,NaN,NaN,13.48,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,3.340000,3.0285,3.06,2.909474,4.65,2.597,2.955122,2.9200,2.359444,4.23,...,5.539444,3.86,NaN,11.258824,3.1929,11.89,7.18,6.45,7.28,2.78
2023-05,3.500000,3.0317,3.04,3.045455,4.50,2.539,2.982838,2.9400,2.342273,3.98,...,5.548636,3.79,NaN,11.940909,3.2059,11.11,7.01,6.36,7.28,2.69
2023-06,3.920000,3.0451,3.05,3.338182,4.35,2.571,3.003048,2.9300,2.376364,3.70,...,5.305500,3.74,NaN,11.939524,3.1647,10.15,7.05,6.22,7.28,2.64
2023-07,4.030000,3.1249,3.14,3.452000,4.15,2.695,3.072380,3.0400,2.456667,3.85,...,5.330500,3.77,NaN,11.724286,3.2688,10.22,7.11,6.33,7.00,2.66


In [11]:
subject = 'LRHUTTTT' # Monthly Unemployment Rate - all persons, s.a.
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, countries_before_subject=False)
df.sort_index()

https://stats.oecd.org/SDMX-JSON/data/KEI/LRHUTTTT.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Missing first:
Türkiye 2005-01
Israel 2012-01
Colombia 2007-01
Missing last:
Done


,Australia,Austria,Belgium,Canada,Czech Republic,Denmark,Finland,France,Germany,Greece,...,Poland,Portugal,Spain,Sweden,Türkiye,United Kingdom,United States,Chile,Israel,Colombia
1999-01,7.118793,4.3,8.9,7.9,7.8,5.0,10.7,12.0,8.9,11.4,...,11.5,5.8,14.8,7.3,NaN,6.2,4.3,8.373607,NaN,NaN
1999-02,7.115217,4.2,9.0,7.9,8.1,5.0,10.6,12.0,8.8,11.4,...,12.0,5.8,14.3,7.0,NaN,6.2,4.4,8.810076,NaN,NaN
1999-03,7.012740,4.2,9.0,7.9,8.6,5.0,10.5,11.8,8.8,11.4,...,12.3,5.9,13.9,7.0,NaN,6.2,4.2,9.311584,NaN,NaN
1999-04,6.979468,4.2,9.0,8.2,8.6,5.0,10.4,11.7,8.7,12.1,...,12.7,6.0,13.6,6.9,NaN,6.1,4.3,10.038610,NaN,NaN
1999-05,7.025072,4.2,9.0,7.9,8.7,5.0,10.3,11.5,8.6,12.0,...,12.9,6.1,13.5,6.7,NaN,6.0,4.2,10.649560,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,3.689922,5.1,5.6,5.0,2.7,5.0,7.0,7.3,2.9,11.6,...,2.8,6.5,12.1,7.1,10.0,4.0,3.4,8.282938,3.605458,10.288648
2023-05,3.566714,4.7,5.6,5.2,2.5,4.9,7.1,7.3,2.9,11.2,...,2.8,6.4,11.9,7.2,9.5,4.2,3.7,8.247289,3.630614,10.096366
2023-06,3.474120,5.1,5.6,5.4,2.7,4.9,7.2,7.3,2.9,11.0,...,2.8,6.3,11.7,7.9,9.6,4.3,3.6,8.364307,3.562542,9.365741
2023-07,3.713433,5.6,5.5,5.5,2.7,5.6,7.4,7.4,2.9,10.8,...,2.8,6.3,11.6,7.0,9.4,NaN,3.5,NaN,3.389526,9.321362


In [12]:
subject = 'CPALTT01' # Consumer Prices - Annual inflation
measure = 'GY' # Growth on the same period of the previous year
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, countries_before_subject=False)
df

https://stats.oecd.org/SDMX-JSON/data/KEI/CPALTT01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GY.M/all?startTime=1999-01&contentType=csv
Missing first:
Argentina 2017-12
Missing last:
Japan 2021-06
Russia 2022-03
Done


,Austria,Belgium,Canada,Czech Republic,Denmark,Finland,France,Germany,Greece,Hungary,...,Brazil,Chile,China (People's Republic of),India,Indonesia,Israel,Russia,Saudi Arabia,South Africa,Colombia
1999-01,0.495185,0.965517,0.660066,3.587444,1.703992,0.496834,0.244279,0.238949,3.696482,9.753136,...,1.645393,3.594859,-1.2,9.375000,70.642504,7.704160,96.918604,-1.657944,8.881200,17.179262
1999-02,0.494234,0.977288,0.659341,2.971768,2.035870,0.760012,0.179326,0.119190,3.689293,9.394904,...,2.242143,3.804064,-1.3,8.638743,53.407429,7.098765,103.242485,-1.854235,8.630610,15.379706
1999-03,0.411522,1.227078,0.987925,2.666667,2.322206,0.787555,0.345269,0.238663,3.399638,9.274907,...,3.016715,4.052286,-1.8,8.947368,45.416667,6.955178,107.584410,-1.361733,7.885714,13.495025
1999-04,0.191624,1.166781,1.648352,2.511078,2.364865,1.064654,0.421133,0.595948,2.805966,9.393232,...,3.345870,4.033417,-1.3,8.355091,37.967631,5.954198,113.064572,-2.301251,7.622298,11.160355
1999-05,0.493285,0.846879,1.533406,2.511078,2.065322,1.365353,0.408267,0.476190,2.372590,8.863680,...,3.140401,3.966940,-2.2,7.712082,30.736258,6.079027,116.711050,-1.391092,7.021518,9.998016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,9.624198,5.604130,4.405874,12.710567,5.282005,7.924528,5.884473,7.169118,3.039219,24.000000,...,4.184788,9.907431,0.1,5.090055,4.329389,5.050505,NaN,2.700160,7.121951,12.822871
2023-05,8.909091,5.197918,3.357472,11.060948,2.928128,6.819794,5.119943,6.102004,2.781513,21.500000,...,3.935829,8.726823,0.2,4.418605,4.004224,4.618474,NaN,2.772035,6.589147,12.360256
2023-06,7.982063,4.154376,2.812296,9.703704,2.464789,6.269261,4.531319,6.375228,1.760478,20.100000,...,3.161474,7.560418,0.0,5.572755,3.520988,4.200000,NaN,2.737671,5.656759,12.129618
2023-07,7.015986,4.136530,3.265839,8.771930,3.133159,6.459540,4.288119,6.165005,2.470822,17.600000,...,3.992389,6.475515,-0.3,7.544265,3.078237,3.363007,NaN,2.313433,4.811321,11.784920


In [13]:
subject = 'B6BLTT02' # Current Account as a % of GDP
measure = 'ST' # Level, ratio or index
freq = 'Q'
df = get_oecd_data(indicator, subject, measure, freq, countries_before_subject=False)
df

https://stats.oecd.org/SDMX-JSON/data/KEI/B6BLTT02.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.Q/all?startTime=1999-Q1&contentType=csv
Missing first:
Belgium 2003-Q1
Greece 2002-Q1
Ireland 2002-Q1
Colombia 2005-Q1
Denmark 2005-Q1
Chile 2003-Q1
Poland 2004-Q1
Netherlands 2003-Q2
Russia 2003-Q1
Saudi Arabia 2006-Q1
Indonesia 2004-Q1
Mexico 2006-Q1
Argentina 2006-Q1
Missing last:
Russia 2021-Q3
Done


,Belgium,Greece,Ireland,United Kingdom,Australia,Canada,Finland,Italy,Colombia,Czech Republic,...,Sweden,Russia,Brazil,India,South Africa,China (People's Republic of),Saudi Arabia,Indonesia,Mexico,Argentina
1999-Q1,NaN,NaN,NaN,-2.767561,-5.629524,-0.146836,1.958241,0.901910,NaN,-2.908802,...,3.003761,NaN,-5.040281,-0.301557,1.636158,0.867666,NaN,NaN,NaN,NaN
1999-Q2,NaN,NaN,NaN,-2.203467,-5.935598,-0.252270,6.392679,1.232603,NaN,-1.883920,...,3.909154,NaN,-4.343579,-0.963408,-1.013187,1.110701,NaN,NaN,NaN,NaN
1999-Q3,NaN,NaN,NaN,-2.207767,-5.656639,0.724169,5.307180,0.680739,NaN,-0.589354,...,3.987418,NaN,-4.093971,-0.696877,-0.243527,3.445788,NaN,NaN,NaN,NaN
1999-Q4,NaN,NaN,NaN,-1.304133,-4.705191,0.360043,5.913340,0.263244,NaN,-3.115128,...,4.306477,NaN,-3.835385,-0.858118,-1.834104,1.597965,NaN,NaN,NaN,NaN
2000-Q1,NaN,NaN,NaN,-1.587838,-4.840895,2.086362,6.547232,0.583550,NaN,-3.747868,...,3.154457,NaN,-3.285335,-1.901961,-0.327737,1.957985,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,-3.913503,-10.431840,11.797633,-4.562042,1.973024,0.684377,-5.272166,-1.632962,-5.934337,-6.314773,...,3.525623,NaN,-2.473490,-2.271273,-1.921027,1.816120,14.535936,0.695421,-2.137519,-1.345756
2022-Q3,-6.825386,-6.976852,3.213750,-2.032066,-0.122180,-1.399075,-3.443724,-4.477286,-7.161150,-7.221750,...,4.376356,NaN,-3.282017,-4.010482,0.032663,2.799411,17.303650,0.575878,-1.424739,-2.059779
2022-Q4,-0.479896,-13.418218,15.499594,-0.388202,1.728127,-1.156630,-2.669222,0.314598,-6.168764,-5.790306,...,6.478738,NaN,-3.847587,-1.681203,-1.619126,1.744591,7.240311,2.195254,-0.680392,0.970018
2023-Q1,-3.982741,1.098207,9.369102,-1.662314,1.941856,-0.452213,-0.911503,0.797863,-3.561610,-0.809308,...,4.770537,NaN,-0.859857,-0.793405,-2.317916,2.808440,6.611104,1.080092,-2.180423,-2.892637


In [14]:
subject = 'PRINTO01' # Industrial production
measure = 'GY' # Growth on the same period of the prevoius year
freq = 'M'
df = get_oecd_data(indicator, subject, measure, freq, countries_before_subject=False)
df

https://stats.oecd.org/SDMX-JSON/data/KEI/PRINTO01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GY.M/all?startTime=1999-01&contentType=csv
Missing first:
Switzerland 2011-10
Missing last:
Russia 2021-12
Done


,Austria,Belgium,Canada,Czech Republic,Denmark,Finland,France,Germany,Greece,Hungary,...,United Kingdom,United States,Brazil,Chile,India,Israel,Russia,Colombia,Euro area (19 countries),Switzerland
1999-01,2.868852,-0.551720,5.455559,-8.702718,1.806589,3.464203,1.415701,0.400534,5.307198,13.445378,...,2.144470,3.638185,-1.062165,-5.357759,5.555856,1.149425,-2.243915,-14.437933,1.245753,NaN
1999-02,0.134589,-2.678550,4.073814,-7.656236,0.858369,2.270148,0.412939,-1.204819,2.622234,5.548038,...,0.783875,4.058844,-4.390297,-4.866265,3.564691,-2.922078,-9.360208,-17.851952,-0.338219,NaN
1999-03,2.673797,-0.309266,2.673290,-9.609728,1.052632,1.791713,1.542135,-1.849406,-5.578570,8.502024,...,1.339286,4.243102,-3.857599,-3.455285,4.666166,-3.626374,-5.861282,-22.563792,-0.225479,NaN
1999-04,4.312668,-0.394179,3.755275,-7.047759,0.333704,1.775805,1.031130,-0.266667,-2.161938,8.010681,...,1.230425,4.122832,-2.995190,-5.374827,5.001758,0.223964,-5.771179,-19.893567,0.112360,NaN
1999-05,4.806409,-0.344492,4.395719,-1.845239,3.543743,2.120536,0.174588,-0.132626,3.566775,4.427083,...,3.636364,4.075301,-3.141081,-3.431228,7.441013,-0.670391,-1.137348,-20.107370,-0.223964,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03,0.236035,-0.235849,1.680033,2.378855,11.076443,3.549784,0.194632,3.198294,-0.323625,-3.821656,...,-1.867414,0.160620,0.335509,-6.079920,NaN,2.700675,NaN,-2.023242,-2.245089,4.934464
2023-04,1.587302,-4.107425,0.719543,1.687389,10.162917,-3.388430,1.514995,2.531646,4.506066,-5.693665,...,-1.313321,0.318011,-0.751102,-1.007666,NaN,2.914798,NaN,-4.755581,0.474383,NaN
2023-05,-2.018634,-3.855232,0.605288,0.612423,6.495468,0.168067,2.238193,1.668405,1.684920,-4.313099,...,-2.061856,-0.033860,-0.039911,-5.360942,NaN,-1.956522,NaN,-3.345041,-2.214022,NaN
2023-06,-3.341103,-4.887821,-0.393338,0.344234,11.483967,-5.800654,-0.262892,0.000000,-3.640777,-5.859066,...,0.757576,-0.782772,0.046400,-2.498001,NaN,-4.288778,NaN,-4.805130,-1.572618,NaN


In [15]:
subject = 'NAEXKP02' # Private Consumption
measure = 'GY' # Growth on the same period of the prevoius year
freq = 'Q'
df = get_oecd_data(indicator, subject, measure, freq, countries_before_subject=False)
df

https://stats.oecd.org/SDMX-JSON/data/KEI/NAEXKP02.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GY.Q/all?startTime=1999-Q1&contentType=csv
Missing first:
India 2012-Q2
Russia 2004-Q1
Colombia 2006-Q1
Missing last:
Russia 2021-Q3
Done


,Australia,Austria,Belgium,Canada,Czech Republic,Denmark,Finland,France,Germany,Greece,...,United Kingdom,United States,Brazil,Chile,India,Indonesia,Israel,Russia,South Africa,Colombia
1999-Q1,6.490941,2.089170,2.532320,3.576515,3.365542,1.752536,3.912584,3.023039,2.650972,3.051948,...,3.456293,5.651322,-1.001505,-3.043256,NaN,2.188967,4.583693,NaN,0.332252,NaN
1999-Q2,5.495034,3.144290,1.972000,3.362945,3.523816,-0.222539,3.555600,2.821045,3.266317,2.276205,...,4.753388,5.413038,-0.645097,-4.940129,NaN,3.892209,3.121635,NaN,1.241840,NaN
1999-Q3,4.613198,1.538851,1.881689,4.047948,3.645058,-0.513521,3.869121,4.000820,2.008873,3.114917,...,4.130955,5.225906,-0.001787,-0.730095,NaN,7.656950,4.338589,NaN,2.206697,NaN
1999-Q4,5.055635,2.853495,2.075101,4.743512,1.928596,-1.447840,2.093476,3.845444,1.993951,1.424292,...,4.878311,5.244597,3.096496,4.840225,NaN,4.899022,4.721557,NaN,3.167523,NaN
2000-Q1,3.477669,2.979463,2.731866,4.343575,1.268848,-0.019015,3.610196,4.771746,1.646271,2.407057,...,5.156592,5.749823,1.772035,5.015551,NaN,0.806698,5.979092,NaN,4.134476,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,5.295827,8.976685,5.537346,7.926034,-0.130161,-1.462915,3.471761,4.760390,5.264206,8.829624,...,4.826089,2.365147,5.631389,7.478210,20.444865,4.880922,8.262006,NaN,1.153628,14.910579
2022-Q3,11.547891,-0.342331,1.128062,3.138237,-4.543280,-3.252893,0.985929,-0.030272,1.605397,5.481078,...,1.633502,2.192283,4.540901,-3.414986,7.642047,5.212993,5.749802,NaN,4.113985,9.129451
2022-Q4,5.198754,2.175020,2.064336,3.039665,-5.258664,-4.918704,-0.211533,-0.883391,0.512529,3.957130,...,1.648990,1.668982,4.345940,-4.409712,1.550136,5.059106,4.379384,NaN,1.886286,3.922484
2023-Q1,3.562526,1.478850,3.302218,3.756974,-5.322971,-1.067334,0.668130,0.410875,-0.368062,2.496861,...,0.257922,2.366445,3.602226,-7.135776,2.802027,5.027742,2.841824,NaN,1.047456,3.055144


## Composite Leading Indicators

In [16]:
indicator = 'MEI_CLI' # Composite Leading Indicators (MEI) 
subject = 'LOLITOAA' # Amplitude adjusted (CLI)
freq = 'M'
df = get_oecd_data(indicator, subject, measure, freq, countries_before_subject=False, with_measure=False)
df

https://stats.oecd.org/SDMX-JSON/data/MEI_CLI/LOLITOAA.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.M/all?startTime=1999-01&contentType=csv
Missing first:
Missing last:
Done


,Australia,Canada,France,Germany,Italy,Japan,Korea,Mexico,Spain,Türkiye,United Kingdom,United States,Brazil,China (People's Republic of),India,Indonesia,South Africa
1999-01,100.33960,99.04568,99.87450,99.32185,99.53897,98.08029,100.97330,99.50538,103.49350,93.92188,99.92517,100.22600,97.41353,99.40079,99.60150,93.44679,97.07583
1999-02,100.44010,99.32136,99.83424,99.29385,99.54351,98.48177,101.89580,99.69563,103.42340,94.92348,100.41620,100.43870,97.70880,99.54219,100.02570,94.28552,97.24391
1999-03,100.52450,99.55447,99.86840,99.32334,99.56295,98.87955,102.81500,99.86613,103.31960,95.96504,100.91410,100.62990,98.10017,99.75793,100.49540,95.19625,97.49399
1999-04,100.60160,99.73008,100.00550,99.41097,99.61215,99.25786,103.66550,100.03850,103.20750,96.85421,101.34050,100.79730,98.55508,100.01600,101.00190,96.20809,97.81783
1999-05,100.66890,99.85417,100.23400,99.56577,99.73546,99.59679,104.40480,100.21470,103.11400,97.49230,101.66880,100.94080,99.02649,100.26520,101.40340,97.27637,98.19668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,98.78822,97.52503,98.56210,99.58736,98.34950,100.08740,98.71696,100.49590,99.42432,100.28580,99.22129,98.89024,99.51808,100.10550,99.68430,97.78961,99.92047
2023-05,98.71284,97.51325,98.53325,99.52377,98.33059,100.13520,98.85375,100.53540,99.52039,100.30800,99.50819,98.97022,99.64621,100.43670,99.74212,97.83102,99.73529
2023-06,98.64871,97.52862,98.51160,99.41216,98.30795,100.17440,98.99844,100.58880,99.61332,100.23180,99.72255,99.08180,99.78266,100.78220,99.81134,97.91052,99.57146
2023-07,98.60187,97.56876,98.49852,99.29068,98.27803,100.20680,99.12750,100.68190,99.69845,100.09870,99.88969,99.20307,99.89719,101.14130,99.88723,97.98292,99.42410


In [17]:
subject = 'BSCICP03' # OECD Standardised Bussiness Confidence Indicator, Amplitude adjusted (Long term average=100), sa
df = get_oecd_data(indicator, subject, measure, freq, countries_before_subject=False, with_measure=False)
df

https://stats.oecd.org/SDMX-JSON/data/MEI_CLI/BSCICP03.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.M/all?startTime=1999-01&contentType=csv
Missing first:
Chile 2003-11
China (People's Republic of) 2000-02
India 2000-05
Indonesia 2002-03
Missing last:
Russia 2022-01
Done


,Australia,Austria,Belgium,Canada,Czech Republic,Denmark,Finland,France,Germany,Greece,...,Brazil,Chile,China (People's Republic of),India,Indonesia,Russia,South Africa,Euro area (19 countries),Israel,Colombia
1999-01,99.88649,98.57338,98.47441,100.00960,94.73983,96.24296,98.07703,99.90664,99.07100,101.1002,...,96.62431,NaN,NaN,NaN,NaN,99.716455,97.36156,98.99956,97.63659,90.59782
1999-02,99.79908,98.49879,98.74121,100.14230,94.91841,96.50874,98.23219,99.86806,98.89312,100.9418,...,96.79649,NaN,NaN,NaN,NaN,100.154733,97.28713,98.91740,97.84121,90.41037
1999-03,99.93929,98.36891,99.04760,100.27130,94.99623,96.75919,98.49326,99.78703,98.79162,100.9921,...,97.00102,NaN,NaN,NaN,NaN,100.682359,97.24184,98.90366,98.11553,90.56648
1999-04,100.18510,98.20333,99.52825,100.43320,95.71694,96.92786,98.93379,99.87154,98.76231,101.2359,...,97.33565,NaN,NaN,NaN,NaN,101.178524,97.27150,98.96791,98.41988,91.26705
1999-05,100.37760,98.15255,99.88451,100.65620,96.74817,97.17252,99.16238,100.09060,98.80769,101.2700,...,97.92552,NaN,NaN,NaN,NaN,101.719888,97.36821,99.13593,98.70652,92.18592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,99.76989,99.12766,98.73976,98.02653,98.88395,99.38604,98.02242,99.91919,100.58270,101.8967,...,99.42391,95.50326,98.46265,NaN,99.04103,NaN,99.02563,100.32200,102.32660,99.50489
2023-05,99.81442,98.65003,98.37148,NaN,98.06341,100.02480,97.76524,99.81200,100.21450,101.7273,...,99.37453,95.46212,97.78954,NaN,99.05051,NaN,98.75212,99.91407,102.37820,99.06139
2023-06,NaN,98.37650,98.03333,NaN,97.37190,100.39650,97.42224,99.73529,99.78830,101.7869,...,99.30090,95.89169,97.59743,NaN,99.06405,NaN,98.46155,99.51776,102.39920,98.75256
2023-07,NaN,98.13904,97.78645,NaN,97.30465,100.48440,97.10443,99.51942,99.34898,101.8207,...,99.14629,96.50355,97.74020,NaN,NaN,NaN,NaN,99.16720,102.33610,NaN


In [18]:
subject = 'CSCICP03' # OECD Standardised Consumer Confidence Indicator, Amplitude adjusted (Long term average=100), sa
df = get_oecd_data(indicator, subject, measure, freq, countries_before_subject=False, with_measure=False)
df

https://stats.oecd.org/SDMX-JSON/data/MEI_CLI/CSCICP03.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.M/all?startTime=1999-01&contentType=csv
Missing first:
Mexico 2001-04
Poland 2001-05
Türkiye 2004-01
Indonesia 2001-04
Israel 2011-03
Colombia 2001-11
Chile 2002-03
India 2012-08
Missing last:
India 2020-09
Done


,Australia,Austria,Belgium,Czech Republic,Denmark,Finland,France,Germany,Greece,Hungary,...,Brazil,China (People's Republic of),Indonesia,South Africa,Euro area (19 countries),Israel,Russia,Colombia,Chile,India
1999-01,101.25230,100.19210,100.99390,96.81049,99.30415,100.85740,101.25410,100.60430,102.41200,100.58040,...,98.04639,98.47530,NaN,99.01440,101.91290,NaN,90.73000,NaN,NaN,NaN
1999-02,101.55030,100.21500,101.01800,96.45536,99.39217,100.82080,101.21350,100.49530,102.44690,100.56310,...,97.55905,98.57423,NaN,99.16352,101.77710,NaN,91.17000,NaN,NaN,NaN
1999-03,101.57250,100.27560,100.95500,96.24995,99.56536,100.95870,101.09430,100.28060,102.52370,100.35150,...,97.07430,98.59915,NaN,99.31396,101.53620,NaN,91.50000,NaN,NaN,NaN
1999-04,101.54850,100.42580,100.82060,96.30946,99.71352,101.22990,101.00480,100.05560,102.75030,100.00990,...,96.94064,98.66280,NaN,99.43877,101.31790,NaN,91.72000,NaN,NaN,NaN
1999-05,101.45660,100.55430,100.54670,96.39417,99.87493,101.53040,101.04590,99.90978,102.91550,99.80379,...,97.21391,98.79964,NaN,99.53185,101.22730,NaN,91.87000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,97.84637,97.13659,99.33556,98.23370,98.21471,97.27126,97.32153,98.86586,98.90280,98.66436,...,100.06050,93.34354,101.6381,96.60242,97.97221,97.92220,99.31689,95.16866,95.58919,NaN
2023-05,97.76232,97.19421,99.29306,98.28234,98.59695,97.45316,97.77294,98.99891,99.50943,98.66811,...,100.36550,NaN,101.7131,96.41818,98.18236,97.88658,99.61455,95.77176,95.79842,NaN
2023-06,97.71120,97.33972,99.37369,98.44428,98.85841,97.52588,98.19633,98.98434,99.99138,98.77574,...,100.80830,NaN,101.6690,96.29654,98.40149,97.80249,NaN,96.42462,96.05412,NaN
2023-07,97.74561,97.45644,99.56100,98.81435,98.94753,97.58981,98.43741,98.88734,100.13840,98.92316,...,101.25640,NaN,101.5384,NaN,98.54996,97.97392,NaN,96.80355,96.21217,NaN
